# Identifying "sparse columns" --- columns without predictive information

We use the following data set
https://www.kaggle.com/datasets/ashrafkhan94/oil-spill

In [1]:
import pandas as pd
df = pd.read_csv('oil-spill.csv', header=None)

In [2]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1,2558,1506.09,456.63,90,6395000.0,40.88,7.89,29780.0,0.19,...,2850.00,1000.00,763.16,135.46,3.73,0,33243.19,65.74,7.95,1
1,2,22325,79.11,841.03,180,55812500.0,51.11,1.21,61900.0,0.02,...,5750.00,11500.00,9593.48,1648.80,0.60,0,51572.04,65.73,6.26,0
2,3,115,1449.85,608.43,88,287500.0,40.42,7.34,3340.0,0.18,...,1400.00,250.00,150.00,45.13,9.33,1,31692.84,65.81,7.84,1
3,4,1201,1562.53,295.65,66,3002500.0,42.40,7.97,18030.0,0.19,...,6041.52,761.58,453.21,144.97,13.33,1,37696.21,65.67,8.07,1
4,5,312,950.27,440.86,37,780000.0,41.43,7.03,3350.0,0.17,...,1320.04,710.63,512.54,109.16,2.58,0,29038.17,65.66,7.35,0


In [2]:
df.shape

(937, 50)

### Count unique values in each column

In [3]:
df.nunique(axis=0)#.head(10)

0     238
1     297
2     927
3     933
4     179
5     375
6     820
7     618
8     561
9      57
10    577
11     59
12     73
13    107
14     53
15     91
16    893
17    810
18    170
19     53
20     68
21      9
22      1
23     92
24      9
25      8
26      9
27    308
28    447
29    392
30    107
31     42
32      4
33     45
34    141
35    110
36      3
37    758
38      9
39      9
40    388
41    220
42    644
43    649
44    499
45      2
46    937
47    169
48    286
49      2
dtype: int64

### Don't do it this way!!

In [5]:
counts = df.nunique()
[i for i,v in enumerate(counts) if v == 1]

[22]

## How should you do it?

In [4]:
df.columns[df.nunique() == 1]

Int64Index([22], dtype='int64')

## Calculate the percentage of unique values in each column

In [9]:
df.nunique()/len(df)*100

0      25.400213
1      31.696905
2      98.932764
3      99.573106
4      19.103522
5      40.021345
6      87.513340
7      65.955176
8      59.871932
9       6.083244
10     61.579509
11      6.296692
12      7.790822
13     11.419424
14      5.656350
15      9.711846
16     95.304162
17     86.446105
18     18.143010
19      5.656350
20      7.257204
21      0.960512
22      0.106724
23      9.818570
24      0.960512
25      0.853789
26      0.960512
27     32.870864
28     47.705443
29     41.835646
30     11.419424
31      4.482391
32      0.426894
33      4.802561
34     15.048026
35     11.739594
36      0.320171
37     80.896478
38      0.960512
39      0.960512
40     41.408751
41     23.479189
42     68.729989
43     69.263607
44     53.255069
45      0.213447
46    100.000000
47     18.036286
48     30.522946
49      0.213447
dtype: float64

## Grotesque

In [10]:
counts = df.nunique()
[i for i,v in enumerate(counts) if (float(v)/df.shape[0]*100) < 1]

[21, 22, 24, 25, 26, 32, 36, 38, 39, 45, 49]

## Divine ...

In [7]:
list(df.columns[df.nunique()/len(df) < 0.01])

[21, 22, 24, 25, 26, 32, 36, 38, 39, 45, 49]

## Using Variance Threshold

$s^2 = \frac{\sum_1^N(x_i - \mu)}{N-1}$

In [12]:
from sklearn.feature_selection import VarianceThreshold

df = pd.read_csv('oil-spill.csv', header=None)

data = df.values

X = data[:, :-1]
y = data[:, -1]

print(X.shape, y.shape)

transform = VarianceThreshold()

X_sel = transform.fit_transform(X)
print(X_sel.shape)

(937, 49) (937,)
(937, 48)


In [13]:
df.columns[:-1][~transform.get_support()]

Int64Index([22], dtype='int64')

In [14]:
transform = VarianceThreshold(threshold=0.05)
transform.fit(X)

VarianceThreshold(threshold=0.05)

In [15]:
[i for i, b in enumerate(transform.get_support()) if not b]

[9, 11, 12, 13, 14, 15, 19, 20, 22, 30, 32, 36]

In [16]:
df.columns[:-1][~transform.get_support()]

Int64Index([9, 11, 12, 13, 14, 15, 19, 20, 22, 30, 32, 36], dtype='int64')

In [17]:
from numpy import arange
import altair as alt

df = pd.read_csv('oil-spill.csv', header=None)

data = df.values
X = data[:, :-1]
y = data[:, -1]

print(X.shape, y.shape)

thresholds = arange(0.0, 0.55, 0.05)

results = []
for t in thresholds:
    
    transform = VarianceThreshold(threshold=t)
    
    X_sel = transform.fit_transform(X)
    rows, cols = X_sel.shape
    n_features = cols
    print('Threshold=%.2f, Features=%d' % (t, n_features))
    
    results.append(n_features)
    
d2 = pd.DataFrame({'threshold': thresholds, 'n_features': results})
alt.Chart(d2).mark_line().encode(
    x='threshold',
    y='n_features')

(937, 49) (937,)
Threshold=0.00, Features=48
Threshold=0.05, Features=37
Threshold=0.10, Features=36
Threshold=0.15, Features=35
Threshold=0.20, Features=35
Threshold=0.25, Features=35
Threshold=0.30, Features=35
Threshold=0.35, Features=35
Threshold=0.40, Features=35
Threshold=0.45, Features=33
Threshold=0.50, Features=31


alt.Chart(...)